In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Function to convert price string into a float/int
def convert_price(price_str):
    # Remove "Rp" and "." then convert to int
    return int(price_str.replace("Rp", "").replace(".", "").strip())

targets = ['clear shampoo', 'sunsilk shampoo', 'lifebuoy', 'tresemme shampoo', 'ponds',
           'glow lovely', 'vaseline', 'pepsodent pasta gigi',
           'closeup', 'lifebuoy sabun mandi', 'lux botanicals', 'rexona', 'axe', 'molto', 'sunlight',
           'wipol', 'vixal', 'royco', 'bango kecap', 'sariwangi',  'buavita',
           'head shoulders', 'pantene shampoo', 'zinc', 'garnier', 'nivea',
           'marina', 'ciptadent pasta gigi', 'nuvo', 'giv', 'posh men body spray',
           'soklin softergent', 'soklin liquid', 'soklin pewangi', 'downy pelembut',
           'mama lemon', 'supersol', 'yuri porstex', 'masako penyedap rasa ayam',
           'sedaap kecap', 'abc kecap manis', 'sosro teh', 'sosro teh asli', 'tong tji',
           'country choice', 'citra', 'superpel']


In [2]:
# url = f"https://www.klikindomaret.com/search/?key=sunlight"

# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
# }

# response = requests.get(url, headers=headers)
# print(response.cookies)

# soup = BeautifulSoup(response.text, "html.parser")
# # print(soup.select_one('.produk .rightside .box-item.wrp-noFilter'))
# if not soup.select_one('.produk .rightside .box-item.wrp-noFilter'):
#     print(f"No result for ")

In [3]:
promo_sku = pd.DataFrame(columns=["productName", "basePrice", "finalPrice", "discountPercent"])
# targets = ['teh bendera']
for target in targets:
    url = f"https://www.klikindomaret.com/search/?key={target}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)
    # print(response.cookies)

    soup = BeautifulSoup(response.text, "html.parser")
    
    #Check if the search result is empty
    if not soup.select_one('.produk .rightside .box-item.wrp-noFilter'):
        print(f"No result for {target}")
        continue
    products = soup.find_all("div", class_="each-item")
    # List to hold the product data
    products_data = []
    
    # Loop through the products and extract the title and price
    for product in products:
        # Extract the title
        title = product.find("div", class_="title")
        if title is not None:
            title_text = title.text.strip()
            # Extract the price
            final_price = product.find("span", class_="normal price-value")
            if final_price:
                final_price = convert_price(final_price.text.strip())
            
            base_price = product.find("span", class_="strikeout disc-price")
            discount_percentage = product.find("span", class_="discount")
            
            if base_price and discount_percentage:
                base_price = convert_price(base_price.text.strip().split("\n")[-1])
                discount_percentage = float(discount_percentage.text.strip().replace("%", "").strip())
            else:
                base_price = final_price
                discount_percentage = 0
            
            
        products_data.append([title_text, base_price, final_price, discount_percentage])
    # print(products_data)
            
    promo_sku = pd.concat([promo_sku, pd.DataFrame(products_data, columns=["productName", "basePrice", "finalPrice", "discountPercent"])], ignore_index=True)

promo_sku.drop_duplicates(subset=["productName"], inplace=True)
promo_sku


No result for country choice


,productName,basePrice,finalPrice,discountPercent
0,Clear Shampoo Anti Ketombe Dan Rambut Rontok 1...,34500,34500,0
1,Clear Shampoo Anti Dandruff Menthol Segar 400mL,67600,67600,0
2,Clear Shampoo Anti Dandruff Fresh Cool Lemon 1...,34500,34500,0
3,Clear Shampoo Anti Dandruff Menthol Segar 70mL,15200,15200,0
4,Clear Shampoo Anti Dandruff Fresh Apple 160mL,34500,34500,0
...,...,...,...,...
1314,Citra Sakura Fair Uv Powder Cream 40G,55600,55600,0.0
1315,Citra Pelembab Wajah Hazeline Pearly White Uv ...,55600,55600,0.0
1333,Super Pell Pembersih Lantai Anti-Bac Citrus 500mL,9900,9900,0
1334,Super Pell Pembersih Lantai Refill Fresh Apple...,16700,16700,0


In [4]:
from datetime import datetime

file_name = f"../indomaret/INDOMARET_{datetime.now().strftime('%y%m%d')}.xlsx"
promo_sku.to_excel(file_name,index=False)